# 1. Introducción

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

# 2. Objetivo del estudio

Este análisis busca examinar los datos de libros, autores, editoriales, calificaciones y reseñas de usuarios
para construir una propuesta de valor competitiva en el mercado de aplicaciones para lectores.

Utilizaremos SQL para responder preguntas clave sobre la base de datos y extraer insights relevantes.


# 3. Análisis

## 3.1 Conexión a la base de datos

In [1]:
# Importar librerías
import pandas as pd
from sqlalchemy import create_engine

# Configuración de la conexión
db_config = {
    'user': 'practicum_student',
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,
    'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'], db_config['pwd'],
    db_config['host'], db_config['port'],
    db_config['db']
)

# Crear el motor de conexión
engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

## 3.2 Exploración de las tablas

In [2]:
# Imprimir 5 filas de cada tabla
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    print(f"--- {table.upper()} ---")
    display(pd.read_sql(
        f"SELECT * FROM {table} LIMIT 5", con=engine))

--- BOOKS ---


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


--- AUTHORS ---


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


--- PUBLISHERS ---


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


--- RATINGS ---


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


--- REVIEWS ---


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## 3.3 Libros publicados después del 1 de enero del 2000

In [3]:
query = """
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
"""
pd.read_sql(query, con=engine)

,books_after_2000
0,819


## 3.4 Número de reseñas y calificación promedio por libro

In [4]:
query = """
SELECT
    b.book_id,
    b.title,
    COUNT(DISTINCT rv.review_id) AS total_reviews,
    ROUND(AVG(rt.rating), 2) AS avg_rating
FROM books b
LEFT JOIN reviews rv ON b.book_id = rv.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY avg_rating DESC;
"""
pd.read_sql(query, con=engine)

,book_id,title,total_reviews,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


## 3.5 Editorial con más libros (> 50 páginas)

In [5]:
query = """
SELECT p.publisher, COUNT(*) AS book_count
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY book_count DESC
LIMIT 1;
"""
pd.read_sql(query, con=engine)

,publisher,book_count
0,Penguin Books,42


## 3.6 Autor con mejor calificación promedio (≥50 calificaciones)

In [6]:
# query = """
# SELECT a.author, ROUND(AVG(r.rating), 2) AS avg_rating, COUNT(r.rating) AS total_ratings
# FROM ratings r
# JOIN books b ON r.book_id = b.book_id
# JOIN authors a ON b.author_id = a.author_id
# GROUP BY a.author
# HAVING COUNT(r.rating) >= 50
# ORDER BY avg_rating DESC
# LIMIT 1;
# """

query = """
WITH books_with_enough_ratings AS (
    SELECT book_id
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(*) >= 50
)
SELECT a.author, ROUND(AVG(r.rating), 2) AS avg_rating
FROM ratings r
JOIN books_with_enough_ratings bwr ON r.book_id = bwr.book_id
JOIN books b ON r.book_id = b.book_id
JOIN authors a ON b.author_id = a.author_id
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 1;
"""
pd.read_sql(query, con=engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


## 3.7 Promedio de reseñas entre usuarios que calificaron >50 libros

In [7]:
query = """
WITH reviewers AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
)
SELECT ROUND(AVG(review_count), 2) AS avg_reviews
FROM (
    SELECT r.username, COUNT(rv.review_id) AS review_count
    FROM reviewers r
    LEFT JOIN reviews rv ON r.username = rv.username
    GROUP BY r.username
) AS user_reviews;
"""
pd.read_sql(query, con=engine)

,avg_reviews
0,24.33


# 4. Conclusiones

- Se han publicado 819 libros desde el año 2000.
- El libro con mayor calificación es "Arrows of the Queen (Heralds of Valdemar #1)", con una media de 5.0 basado en 2 reseñas.
- La editorial con más libros con más de 50 páginas es Penguin Books
- El autor con mejor calificación promedio (≥50 ratings) es J.K. Rowling/Mary GrandPré con un promedio de 4.29 puntos.
- Los usuarios que calificaron más de 50 libros escriben en promedio 24 reseñas.

Estos resultados pueden orientar el desarrollo de funcionalidades como recomendaciones, rankings por autor/editorial, y segmentación por usuarios activos.